<a href="https://colab.research.google.com/github/BNarayanaReddy/CS7015/blob/main/Chapter12_SequentialModels_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# LSTM training from scratch
import torch
import numpy as np
import torch.nn as nn
import transformers
from huggingface_hub import notebook_login
import matplotlib.pyplot as plt


In [2]:
data = """Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, ‘and what is the use of a book,’ thought Alice ‘without pictures or conversation?' So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her. There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, ‘Oh dear! Oh dear! I shall be late!’ (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat-pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit-hole under the hedge.
"""

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
emb_model = AutoModel.from_pretrained('google-bert/bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
tokens = tokenizer.tokenize(data)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

In [6]:
len(tokens) # vocab size

302

In [7]:
# convert to embeddings
token_ids = torch.tensor(token_ids).unsqueeze(dim=0)

token_ids.shape

input_embeddings = emb_model(token_ids)

input_embeddings.last_hidden_state

tensor([[[ 0.0040, -0.0719, -0.1450,  ...,  0.0961,  0.2868,  0.7926],
         [-0.8736, -0.1772, -0.1495,  ...,  0.3733,  0.5888,  0.8709],
         [-1.3055,  0.1648, -0.1523,  ...,  0.4047,  0.0472,  0.4095],
         ...,
         [-0.1763, -0.5055, -0.0862,  ...,  0.1326, -0.3608,  0.6157],
         [ 0.0141,  0.1619,  0.0260,  ...,  0.1147,  0.0109,  0.6323],
         [-0.1190, -0.1171, -0.2277,  ...,  0.9868,  0.1715,  0.5009]]],
       grad_fn=<NativeLayerNormBackward0>)

In [8]:
input_embeddings = input_embeddings.last_hidden_state.detach()

input_embeddings.requires_grad

False

In [9]:
# build the lstm cell
class LSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super().__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.output_dim = output_dim

    self.forget = nn.Linear(input_dim + hidden_dim, hidden_dim)
    self.input = nn.Linear(input_dim + hidden_dim, hidden_dim)
    self.out_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)
    self.cell = nn.Linear(input_dim + hidden_dim, hidden_dim)

    self.sigmoid = nn.Sigmoid()
    self.tanh = nn.Tanh()

    self.out = nn.Linear(hidden_dim, output_dim)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x, hidden_state, cell_state):
    input_prev_h = torch.cat((x, hidden_state), dim=1)

    forget = self.forget(input_prev_h)
    forget = self.sigmoid(forget)

    input_gate = self.input(input_prev_h)
    input_gate = self.sigmoid(input_gate)

    out_gate = self.out_gate(input_prev_h)
    out_gate = self.sigmoid(out_gate)

    cell_state_t = self.cell(input_prev_h)
    cell_state_t = self.tanh(cell_state_t)

    cell_state = (cell_state_t * input_gate) + (cell_state * forget)
    hidden_state = self.tanh(cell_state) * out_gate

    out = self.out(hidden_state)
    out_probs = self.softmax(out)

    return out_probs, hidden_state, cell_state

In [10]:
ce_loss = nn.CrossEntropyLoss()

In [11]:
def fit_lstm_model(vocab_size, hidden_dim=256, input_emb_dim = 768, epochs = 10, lr = 1e-3, eps = 1e-8):

  lstm_model = LSTM(input_emb_dim, hidden_dim, vocab_size)
  optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr, eps=eps)

  for epoch in range(1, epochs):
    optimizer.zero_grad()

    # randomly mask the input embeddings
    output_idx = np.random.randint(0, vocab_size)
    input_embed = input_embeddings.clone()
    input_embed[:, output_idx] = torch.randn(1, input_emb_dim)

    hidden_state = torch.zeros(1, hidden_dim) # zero down the states per each text sequence or sentence
    cell_state = torch.zeros(1, hidden_dim)

    for tensor in input_embed[0]:
      # teacher forced training
      output, hidden_state, cell_state = lstm_model(tensor.unsqueeze(dim=0), hidden_state.detach(), cell_state.detach())

    # After applying forward of the sequences

    loss = ce_loss(output, torch.tensor(output_idx).unsqueeze(0))

    loss.backward()
    optimizer.step()

    if epoch % 1 == 0:
      print(f"Epoch: {epoch}, Loss: {loss.item()}, Masked Token = {tokens[output_idx]} Output Token: {tokens[torch.argmax(output)]}")

  return output, loss.item()



In [12]:
fit_lstm_model(len(tokens), epochs = 100)

Epoch: 1, Loss: 5.710581302642822, Masked Token = dear Output Token: ,
Epoch: 2, Loss: 5.710561275482178, Masked Token = as Output Token: dear
Epoch: 3, Loss: 5.710864543914795, Masked Token = , Output Token: dear
Epoch: 4, Loss: 5.71127986907959, Masked Token = trouble Output Token: dear
Epoch: 5, Loss: 5.710628986358643, Masked Token = ; Output Token: dear
Epoch: 6, Loss: 5.710662841796875, Masked Token = nor Output Token: dear
Epoch: 7, Loss: 5.710498332977295, Masked Token = sister Output Token: dear
Epoch: 8, Loss: 5.711270332336426, Masked Token = with Output Token: dear
Epoch: 9, Loss: 5.70947265625, Masked Token = the Output Token: dear
Epoch: 10, Loss: 5.709877967834473, Masked Token = on Output Token: dear
Epoch: 11, Loss: 5.71036958694458, Masked Token = a Output Token: dear
Epoch: 12, Loss: 5.709682941436768, Masked Token = of Output Token: dear
Epoch: 13, Loss: 5.711774826049805, Masked Token = the Output Token: dear
Epoch: 14, Loss: 5.710801601409912, Masked Token = never

(tensor([[0.0014, 0.0006, 0.0012, 0.0084, 0.0063, 0.0037, 0.0040, 0.0066, 0.0057,
          0.0045, 0.0008, 0.0066, 0.0189, 0.0193, 0.0012, 0.0046, 0.0010, 0.0011,
          0.0010, 0.0040, 0.0016, 0.0144, 0.0009, 0.0048, 0.0012, 0.0038, 0.0016,
          0.0011, 0.0012, 0.0014, 0.0049, 0.0014, 0.0015, 0.0012, 0.0158, 0.0013,
          0.0010, 0.0119, 0.0011, 0.0009, 0.0010, 0.0010, 0.0008, 0.0010, 0.0012,
          0.0008, 0.0014, 0.0127, 0.0008, 0.0059, 0.0070, 0.0011, 0.0009, 0.0041,
          0.0039, 0.0079, 0.0011, 0.0006, 0.0009, 0.0010, 0.0014, 0.0009, 0.0017,
          0.0008, 0.0008, 0.0009, 0.0024, 0.0145, 0.0011, 0.0056, 0.0083, 0.0011,
          0.0008, 0.0009, 0.0012, 0.0010, 0.0011, 0.0172, 0.0009, 0.1644, 0.0013,
          0.0011, 0.0013, 0.0008, 0.0016, 0.0076, 0.0009, 0.0012, 0.0010, 0.0008,
          0.0012, 0.0009, 0.0011, 0.0052, 0.0008, 0.0013, 0.0011, 0.0012, 0.0012,
          0.0014, 0.0006, 0.0013, 0.0012, 0.0012, 0.0019, 0.0013, 0.0009, 0.0009,
          0.0099